In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import data from ENTSOE

Warning! The data of ENTSOE is actualised every day at 14:00 CEST. The server isn't available at this time +- 5 minutes!

https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=GRAPH&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=04.04.2024%2000:00|CET%7CDAYTIMERANGE&dateTime.endDateTime=04.04.2024%2000:00|CET%7CDAYTIMERANGE&area.values=CTY%7C10Y1001A1001A83F!CTY%7C10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)

In [56]:
# Get the current date
current_date = datetime.now()
# Format the date as dd.mm.yyyy
formatted_date_today = current_date.strftime("%d.%m.%Y")
formatted_date_today = "03.04.2024"


# URL of the webpage containing the table
url = f"https://transparency.entsoe.eu/generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime={formatted_date_today}+00:00|CET|DAYTIMERANGE&dateTime.endDateTime={formatted_date_today}+00:00|CET|DAYTIMERANGE&area.values=CTY|10Y1001A1001A83F!CTY|10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)"

# Send a GET request to fetch the webpage content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables in the HTML
tables = soup.find_all('table')

# Iterate through each table to find the one containing "Biomass"
target_table = None
for table in tables:
    if "Biomass" in table.get_text():
        target_table = table
        break

if target_table:
    # Convert the HTML table to a pandas DataFrame
    df = pd.read_html(str(target_table))[0]

    # Replace "n/e" with NaN
    df.replace('n/e', pd.NA, inplace=True)

    # Print the DataFrame
    # print(df)
    df.to_csv("../../data/carbon_footprint_data.csv", sep=';', header=True,index=False)
    
else:
    print("No table containing 'Biomass' found on the webpage.")

ConnectionError: HTTPSConnectionPool(host='transparency.entsoe.eu', port=443): Max retries exceeded with url: /generation/r2/actualGenerationPerProductionType/show?name=&defaultValue=false&viewType=TABLE&areaType=CTY&atch=false&datepicker-day-offset-select-dv-date-from_input=D&dateTime.dateTime=03.04.2024+00:00%7CCET%7CDAYTIMERANGE&dateTime.endDateTime=03.04.2024+00:00%7CCET%7CDAYTIMERANGE&area.values=CTY%7C10Y1001A1001A83F!CTY%7C10Y1001A1001A83F&productionType.values=B01&productionType.values=B02&productionType.values=B03&productionType.values=B04&productionType.values=B05&productionType.values=B06&productionType.values=B07&productionType.values=B08&productionType.values=B09&productionType.values=B10&productionType.values=B11&productionType.values=B12&productionType.values=B13&productionType.values=B14&productionType.values=B20&productionType.values=B15&productionType.values=B16&productionType.values=B17&productionType.values=B18&productionType.values=B19&dateTime.timezone=CET_CEST&da

In [ ]:
df = pd.read_csv("../../data/carbon_footprint_data.csv", sep=";",header=1)
df = df.iloc[3:]
# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)
df[['start', 'end']] = df['MTU'].str.split(' - ', expand=True)


# Get the current date and time in CET timezone
current_datetime_cet = datetime.now(pytz.timezone('Europe/Paris'))

# Extract the date part
current_date = current_datetime_cet.date()

# Convert the "start" column to datetime format with the current date
df['start_datetime'] = pd.to_datetime(current_date.strftime("%Y-%m-%d") + ' ' + df['start'])
df['start_datetime'] = df['start_datetime'].dt.tz_localize('Europe/Paris')

# Convert the datetime to Unix timestamp
df['start_unix_timestamp'] = df['start_datetime'].astype(int) / 10**9  # Convert nanoseconds to seconds

# Convert the "start" column to datetime format with the current date
df['end_datetime'] = pd.to_datetime(current_date.strftime("%Y-%m-%d") + ' ' + df['end'])
df['end_datetime'] = df['end_datetime'].dt.tz_localize('Europe/Paris')

# Convert the datetime to Unix timestamp
df['end_unix_timestamp'] = df['end_datetime'].astype(int) / 10**9  # Convert nanoseconds to seconds

df.to_csv("../../data/carbon_footprint_data.csv", sep=';', header=True,index=False)


KeyError: 'MTU'

# CO2 Äquivalente

https://www.umweltbundesamt.de/sites/default/files/medien/1410/publikationen/2023_05_23_climate_change_20-2023_strommix_bf.pdf

In [ ]:
co2_dict= {
    'Biomass': 230,
    'Fossil Brown coal/Lignite': 1137,
    'Fossil Coal-derived gas': 650,
    'Fossil Gas': 381,
    'Fossil Hard coal': 853,
    'Fossil Oil': 859,
    'Fossil Oil shale': 0,
    'Fossil Peat': 0,
    'Geothermal': 38,
    'Hydro Pumped Storage': 32,
    'Hydro Pumped Storage.1': 32,
    'Hydro Run-of-river and poundage': 32,
    'Hydro Water Reservoir': 32,
    'Marine': 24,
    'Nuclear': 11,
    'Other': 200,
    'Other renewable': 0,
    'Solar': 143,
    'Waste': 0,
    'Wind Offshore': 4,
    'Wind Onshore': 7
}


CO2 Äquivalente Tabelle berechnen

In [ ]:
df["Biomass_co2e"]=df["Biomass"]*co2_dict["Biomass"]
df["Fossil Brown coal/Lignite_co2e"]=df["Fossil Brown coal/Lignite"]*co2_dict["Fossil Brown coal/Lignite"]
df["Fossil Coal-derived gas_co2e"]=df["Fossil Coal-derived gas"]*co2_dict["Fossil Coal-derived gas"]
df["Fossil Gas_co2e"]=df["Fossil Gas"]*co2_dict["Fossil Gas"]
df["Fossil Hard coal_co2e"]=df["Fossil Hard coal"]*co2_dict["Fossil Hard coal"]
df["Fossil Oil_co2e"]=df["Fossil Oil"]*co2_dict["Fossil Oil"]
df["Fossil Oil shale_co2e"]=df["Fossil Oil shale"]*co2_dict["Fossil Oil shale"]
df["Fossil Peat_co2e"]=df["Fossil Peat"]*co2_dict["Fossil Peat"]
df["Geothermal_co2e"]=df["Geothermal"]*co2_dict["Geothermal"]
df["Hydro Pumped Storage_co2e"]=df["Hydro Pumped Storage"]*co2_dict["Hydro Pumped Storage"]
df["Hydro Pumped Storage.1_co2e"]=df["Hydro Pumped Storage.1"]*co2_dict["BiomaHydro Pumped Storage."]
df["Hydro Run-of-river and poundage_co2e"]=df["Hydro Run-of-river and poundage"]*co2_dict["Hydro Run-of-river and poundage"]
df["Hydro Water Reservoir_co2e"]=df["Hydro Water Reservoir"]*co2_dict["Hydro Water Reservoir"]
df["Marine_co2e"]=df["Marine"]*co2_dict["Marine"]
df["Nuclear_co2e"]=df["Nuclear"]*co2_dict["Nuclear"]
df["Other_co2e"]=df["Other"]*co2_dict["Other"]
df["Other renewable_co2e"]=df["Other renewable"]*co2_dict["Other renewable"]
df["Solar_co2e"]=df["Solar"]*co2_dict["Solar"]
df["Waste_co2e"]=df["Waste"]*co2_dict["Waste"]
df["Wind Offshore_co2e"]=df["Wind Offshore"]*co2_dict["Wind Offshore"]
df["Wind Onshore_co2e"]=df["Wind Onshore"]*co2_dict["Wind Onshore"]



KeyError: 'BiomaHydro Pumped Storage.1ss'